Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import cobra
import escher

# Load model

Choose from alternatives

In [ ]:
# Yeast 8
model = cobra.io.read_sbml_model("./models/yeast-GEM-BiGG.xml")

In [ ]:
# Enzyme-constrained Yeast 8, batch
# https://github.com/SysBioChalmers/ecModels/tree/main/ecYeastGEM/model
# This is supposed under CI, i.e.
# automatically re-generated and updated when new models are available.
# This model is based on Yeast8.3.4

# Average enzyme saturation factor (sigma) = 0.5
# Total protein content in the cell [g protein/gDw] (Ptot) = 0.5
# Fraction of enzymes in the model [g enzyme/g protein] (f) = 0.5
# https://github.com/SysBioChalmers/GECKO/blob/main/userData/ecYeastGEM/YeastGEMAdapter.m
model = cobra.io.read_sbml_model("./models/ecYeastGEM_batch.xml")

Show model

In [ ]:
model

# Objective function

In the ecYeast8 (batch) model, the objective function -- growth -- is reaction ID `r_2111`.

This reaction is linked to the biomass reaction, ID `r_4041`.

Here, we also see the stoichiometry.  There are five classes of macromolecules: lipids, proteins, carbohydrates, DNA, and RNA.  And there are two other bulk metabolites: cofactor and ion.

In [ ]:
model.reactions.get_by_id('r_2111')

In [ ]:
model.reactions.get_by_id('r_4041')

Remove bounds on glucose uptake and growth rate

In [ ]:
# (no need because bounds are already unrestricted)
# Unrestrict glucose uptake
model.reactions.get_by_id('r_1714').bounds = (-1000.0, 0)
# Unrestrict oxygen uptake (aerobic)
model.reactions.get_by_id('r_1992').bounds = (-1000.0, 0)
# Unrestrict objective function
model.reactions.get_by_id('r_4041').bounds = (0, 1000.0)

Optimise using (vanilla) FBA

In [ ]:
solution = model.optimize()

In [ ]:
model.summary()

In [ ]:
doubling_time = np.log(2)/solution.fluxes['r_2111']
doubling_time

> Doubling time is predicted to be 1.84 hours.  Experimental data suggests 1.5 hours.

Linear reaction coefficients

In [ ]:
cobra.util.solver.linear_reaction_coefficients(model)

# Change media

Show medium

In [ ]:
model.medium

In [ ]:
for reaction_id in model.medium.keys():
    print(model.reactions.get_by_id(reaction_id).name)

## Pyruvate

Pyruvate exchange is `r_2033`

In [ ]:
model.reactions.get_by_id('r_2033')

Kill glucose uptake, unrestrict pyruvate uptake.

In [ ]:
# kill glucose
model.reactions.get_by_id('r_1714').bounds = (0, 0)
model.reactions.get_by_id('r_1714_REV').bounds = (0, 0)
# unrestrict pyruvate
model.reactions.get_by_id('r_2033').bounds = (-1000.0, 0)

Solve again

In [ ]:
solution = model.optimize()
model.summary()

In [ ]:
doubling_time = np.log(2)/solution.fluxes['r_2111']
doubling_time

> Doubling time is predicted to be 2.68 hours.  Experimental data suggests 3-4 hours.

## Others

Other media of interest, beyond experiments: ethanol, glycerol, ethanol+glycerol, acetyldehyde/acetate.  These have been used in YMC studies, e.g. with synchrony experiments.

Ethanol

In [ ]:
# kill glucose
model.reactions.get_by_id('r_1714').bounds = (0, 0)
model.reactions.get_by_id('r_1714_REV').bounds = (0, 0)
# unrestrict ethanol
model.reactions.get_by_id('r_1761').bounds = (-1000, 0)

Glycerol

In [ ]:
# kill glucose
model.reactions.get_by_id('r_1714').bounds = (0, 0)
model.reactions.get_by_id('r_1714_REV').bounds = (0, 0)
# unrestrict glycerol
model.reactions.get_by_id('r_1808').bounds = (-1000, 0)

Ethanol + glycerol

In [ ]:
# kill glucose
model.reactions.get_by_id('r_1714').bounds = (0, 0)
model.reactions.get_by_id('r_1714_REV').bounds = (0, 0)
# unrestrict ethanol
model.reactions.get_by_id('r_1761').bounds = (-1000, 0)
# unrestrict glycerol
model.reactions.get_by_id('r_1808').bounds = (-1000, 0)

Acetaldehyde

In [ ]:
# kill glucose
model.reactions.get_by_id('r_1714').bounds = (0, 0)
model.reactions.get_by_id('r_1714_REV').bounds = (0, 0)
# unrestrict acetaldehyde
model.reactions.get_by_id('r_1631').bounds = (-1000, 0)

Optimise

In [ ]:
solution = model.optimize()
model.summary()

In [ ]:
doubling_time = np.log(2)/solution.fluxes['r_2111']
doubling_time

> Interpretation: Growth is slower, as expected.  But how do fluxes explain behaviour in YMC?  Inspect redox compounds like NAD(P)H, or flavoprotein-catalysed reaction fluxes?  Of course there is a caveat -- FBA assumes steady-state, but YMC is cyclic.

## Ablation?